### 코드 최신화, 첫 번째 시도

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama  # ChatOllama 유지
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain  # 최신 요약 체인 사용
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ========== ① 문서로드 ========== #

# PDF 파일 로드
loader = PyPDFLoader("testfile.pdf") #현진건 운수좋은날로 테스트
documents = loader.load()
print(f'첫 페이지 내용: {documents[0].page_content[:200]}')

# ========== ② 문서분할 ========== #

# 스플리터 지정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,   # 사이즈
    chunk_overlap=500, # 중첩 사이즈
    separators=["\n\n", "\n", " ", ""],  # 분할기준
)

# 분할 실행
split_docs = text_splitter.split_documents(documents)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

# ========== ③ 체인 생성 ========== #

# ChatOllama 모델
llm = ChatOllama(model="llama3.1:latest")

# 요약 체인 생성
summarize_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",  # 최신 버전에서 사용하는 요약 체인
)

# ========== ④ 실행 결과 ========== #

# 요약 체인 실행
result = summarize_chain.run(split_docs)

# 요약 결과 출력
print(result)


두 번째 시도,
****
**GPT4o :**
#**최신화 경고 반영**: `PyPDFLoader`를 `langchain_community.document_loaders`에서 가져오도록 수정.

- **`run()` 대신 `invoke()` 사용**: 최신 LangChain 버전에서 `run()`이 제거되고 `invoke()`로 대체되었습니다.
- **`token_max` 제거**: `token_max`는 적합한 매개변수가 아니므로 제거했습니다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama  # ChatOllama 유지
from langchain_community.document_loaders import PyPDFLoader  # 최신화 반영
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain  # 최신 요약 체인 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ========== ① 문서로드 ========== #

# PDF 파일 로드
loader = PyPDFLoader("testfile.pdf") #현진건 운수좋은날로 테스트
documents = loader.load()
print(f'첫 페이지 내용: {documents[0].page_content[:200]}')

# ========== ② 문서분할 ========== #

# 스플리터 지정 - 토큰 제한을 고려해 청크 크기 조정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # 모델의 1024 토큰 제한을 고려해 크기 조정
    chunk_overlap=200, # 중복 허용 범위 설정
    separators=["\n\n", "\n", " ", ""],  # 분할 기준
)

# 분할 실행
split_docs = text_splitter.split_documents(documents)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

# ========== ③ 체인 생성 ========== #

# ChatOllama 모델
llm = ChatOllama(model="llama3.1:latest", max_tokens=1024)  # 최대 토큰 수 설정

# 요약 체인 생성
summarize_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",  # 최신 버전에서 사용하는 요약 체인
)

# ========== ④ 실행 결과 ========== #

# 요약 체인 실행
result = summarize_chain.invoke(split_docs)  # run() 대신 invoke() 사용
# summarize_chain.invoke(split_docs,return_intermediate_steps=True)가 기본옵션인듯..

# 요약 결과 출력
print(result)


### 코드 최신화 세 번째 시도

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama  # ChatOllama 유지
from langchain_community.document_loaders import PyPDFLoader  # 최신화 반영
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains.summarize import load_summarize_chain  # 최신 요약 체인 사용
from langchain.schema.runnable import RunnableMap, RunnableSequence  # 수정된 부분



# ========== ① 문서로드 ========== #

# PDF 파일 로드
loader = PyPDFLoader("testfile.pdf") #현진건 운수좋은날로 테스트
documents = loader.load()
print(f'첫 페이지 내용: {documents[0].page_content[:200]}')

# ========== ② 문서분할 ========== #

# 스플리터 지정 - 토큰 제한을 고려해 청크 크기 조정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,   # 모델의 1024 토큰 제한을 고려해 크기 조정
    chunk_overlap=200, # 중복 허용 범위 설정
    separators=["\n\n", "\n", " ", ""],  # 분할 기준
)

# 분할 실행
split_docs = text_splitter.split_documents(documents)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

# ========== ③ 체인 생성 ========== #

# ChatOllama 모델 유지
llm = ChatOllama(model="llama3.1:latest", max_tokens=2048)  # 최대 토큰 수 설정

# 요약할 내용을 한국어로 요청하는 프롬프트 템플릿을 설정
map_template = """다음은 문서 중 일부 내용입니다:
{pages}
이 문서 목록을 기반으로 주요 내용을 **한국어로** 요약해 주세요.
답변:"""

# PromptTemplate 설정
map_prompt = PromptTemplate(template=map_template, input_variables=["pages"])

# ========== ④ RunnableMap 및 RunnableSequence 설정 ========== #

# RunnableMap을 사용하여 프롬프트와 LLM 연결
runnable_map = RunnableMap(
    input={"pages": lambda docs: docs},  # 문서를 입력으로 사용
    steps={"output": map_prompt | llm}   # 프롬프트와 LLM을 연결
)

# 요약 체인 생성 (최신 체인)
summarize_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",  # 최신 버전에서 사용하는 요약 체인
)

# ========== ⑤ 실행 결과 ========== #

# 요약 체인 실행, 중간 단계를 포함한 결과를 반환하도록 설정
result = summarize_chain.invoke(split_docs, return_intermediate_steps=True)

# 요약 결과 출력
print(result)

웹 기반 RAG종합코드(수정됨)

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install chromadb
!pip install transformers

In [ ]:
import os
import warnings
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import unicodedata
from tqdm import tqdm  # tqdm를 사용하여 진행 상황 표시

warnings.filterwarnings("ignore")

def load_files_from_directory(directory_path):
    """디렉토리 내의 모든 PDF 및 TXT 파일을 로드하여 텍스트를 반환합니다."""
    data = []
    files = [f for f in os.listdir(directory_path) if f.endswith(('.pdf', '.txt'))]
    for filename in tqdm(files, desc="Loading Files", unit="file"):
        file_path = os.path.join(directory_path, filename)
        if filename.endswith('.pdf'):
            loader = PyMuPDFLoader(file_path)
            data.extend(loader.load())
        elif filename.endswith('.txt'):
            loader = TextLoader(file_path)
            data.extend(loader.load())
    return data


In [ ]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_community.chat_models import ChatOllama

# 이 줄을:
#from langchain.llms import Ollama
# 다음과 같이 변경:
from langchain_community.llms import Ollama

# 그리고 이 줄을:
#from langchain.embeddings import OllamaEmbeddings
# 다음과 같이 변경:
from langchain_community.embeddings import OllamaEmbeddings

from langchain_core.prompts import PromptTemplate

# BeautifulSoup 파서 설정
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
print(f"분할된 청크의 수: {len(splits)}")

# Ollama 임베딩 초기화 및 벡터스토어 생성
embedding_model = OllamaEmbeddings(model="steamdj/llama3.1-cpu-only:latest")
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = Ollama(model="steamdj/llama3.1-cpu-only:latest", temperature=0.5)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문에 대한 답변 생성
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요."
answer = rag_chain.invoke(question)
print(answer)

# 스트리밍 응답을 원한다면 아래 코드를 사용
# from langchain_teddynote.messages import stream_response
# answer_stream = rag_chain.stream(question)
# stream_response(answer_stream)

## **<모델 파인튜닝>**

https://medium.com/@aifusion.contact/llama-3-1-70b-gpu-requirements-5e292734d36e: llama3.1모델학습 요구사항 관련

## **Fine-Tuning with Llama 2, Bits and Bytes, and QLoRA**

- QLoRA: [Quantized Low Rank Adapters](https://arxiv.org/pdf/2305.14314.pdf)
• 소량의 양자화된 업데이트 가능한 매개변수를 사용하여 LLM(대규모 언어 모델)을 미세 조정하는 방법
• 소규모 매개변수를 모델에 효율적으로 추가하여 다양한 데이터 세트에서 미세 조정 가능
• 필요에 따라 "어댑터"를 모델에 교체하여 사용 가능
- [Bits and Bytes](https://github.com/TimDettmers/bitsandbytes):
• Tim Dettmers가 개발한 패키지
• 최적화, 행렬 곱셈, 양자화를 위한 맞춤형 CUDA 함수의 경량 래퍼 제공
• 모델을 가능한 한 효율적으로 로드하는 데 사용
- [PEFT](https://github.com/huggingface/peft):
• Huggingface에서 제공하는 라이브러리
• 비용 효율적인 미세 조정을 가능하게 하는 다양한 방법 제공
• Kaggle 노트북과 같은 경량 하드웨어에서 특히 유용

https://blog.sionic.ai/finetuning_llama

**파이썬 라이브러리 설치 (버전을 명시할 필요, 라이브러리 구성을 위해 가상환경 새로 구축 필요)**

In [ ]:
pip install bitsandbytes accelerate==0.21.0 scipy tensorboardX peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboardX

**CUDA Library설치**

![image.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/961c8a41-8e92-4022-aaca-e1e87c240949/08aa7b13-b47e-482f-a993-9b219e033fd9/image.png)

**CUDA Library설치**

https://developer.nvidia.com/cuda-downloads

Ubuntu 20.04의 경우

In [ ]:
sudo apt-get install -y nvidia-open

sudo apt-get install -y cuda-drivers

sudo apt install nvidia-cuda-toolkit

wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
wget https://developer.download.nvidia.com/compute/cuda/12.6.1/local_installers/cuda-repo-ubuntu2004-12-6-local_12.6.1-560.35.03-1_amd64.deb
sudo dpkg -i cuda-repo-ubuntu2004-12-6-local_12.6.1-560.35.03-1_amd64.deb
sudo cp /var/cuda-repo-ubuntu2004-12-6-local/cuda-*-keyring.gpg /usr/share/keyrings/
sudo apt-get update
sudo apt-get -y install cuda-toolkit-12-6

**문제점: 12.6, 560버전은 ollama 제대로 작동 안함**

코드1

In [ ]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = "NousResearch/Llama-2-7b-chat-hf"

# instruction 데이터 세트 설정
dataset_name = "mlabonne/guanaco-llama2-1k"

# fine-tuning(미세 조정)을 거친 후의 모델에 부여될 새로운 이름을 지정하는 변수
new_model = "sionic-llama-2-7b-miniguanaco"

# LoRA에서 사용하는 low-rank matrices 어텐션 차원을 정의. 여기서는 64로 설정
# 값이 크면 클수록 더 많은 수정이 이루어지며, 모델이 더 복잡해질 수 있음
lora_r = 64

# LoRA 적용 시 가중치에 곱해지는 스케일링 요소. 여기서는 16으로 설정
# LoRA가 적용될 때 원래 모델의 가중치에 얼마나 영향을 미칠지 결정. 높은 값은 가중치 조정의 강도를 증가시킴
lora_alpha = 16

# Dropout probability for LoRA layers   # LoRA 층에 적용되는 드롭아웃 확률. 여기서는 0.1 (10%)로 설정
lora_dropout = 0.1 # 일부 네트워크 연결을 무작위로 비활성화하여 모델의 강건함에 기여


####### bitsandbytes 파라미터 설정######
###BitsAndBytes:  벡터의 INT8 최대 절대값 양자화 기법을 사용(Meta AI의 라이브러리)

# 4-bit precision 기반의 모델 로드
use_4bit = True

# 4비트 기반 모델에 대한 dtype 계산
bnb_4bit_compute_dtype = "float16"

# 양자화 유형(fp4 또는 nf4)
bnb_4bit_quant_type = "nf4"

# 4비트 기 모델에 대해 중첩 양자화 활성화(이중 양자화)
use_nested_quant = False


######## TrainingArguments 파라미터 설정#########

#모델이 예측한 결과와 체크포인트가 저장될 출력 디렉터리
output_dir = "./results"

# 훈련 에포크 수
num_train_epochs = 1

# fp16/bf16 학습 활성화(A100으로 bf16을 True로 설정)
fp16 = False
bf16 = False

# 훈련용 배치 크기
per_device_train_batch_size = 1

# 평가용 배치 크기
per_device_eval_batch_size = 1

# 그래디언트를 누적할 업데이트 스텝 횟수
gradient_accumulation_steps = 1

# 그래디언트 체크포인트 활성화
gradient_checkpointing = True


# 그래디언트 클리핑을 위한 최대 그래디언트 노름을 설정.
# 그래디언트 클리핑은 그래디언트의 크기를 제한하여 훈련 중 안정성을 높임.
# Maximum gradient normal (그래디언트 클리핑) 0.3으로 설정
max_grad_norm = 0.3

# 초기 학습률 AdamW 옵티마이저
learning_rate = 2e-6

# bias/LayerNorm 가중치를 제외하고 모든 레이어에 적용할 Weight decay 값
weight_decay = 0.001

# 옵티마이저 설정
optim = "paged_adamw_32bit"

# 학습률 스케줄러의 유형 설정, 여기서는 코사인 스케줄러 사용
lr_scheduler_type = "cosine"

# 훈련 스텝 수(num_train_epochs 재정의)
max_steps = -1

# (0부터 learning rate까지) 학습 초기에 학습률을 점진적으로 증가시키 linear warmup 스텝의 Ratio
warmup_ratio = 0.03

# 시퀀스를 동일한 길이의 배치로 그룹화, 메모리 절약 및 훈련 속도를 높임
group_by_length = True

# X 업데이트 단계마다 체크포인트 저장
save_steps = 0

# 매 X 업데이트 스텝 로그
logging_steps = 25


##########  SFT 파라미터 값 설정###########
# 최대 시퀀스 길이 설정
max_seq_length = None

# 동일한 입력 시퀀스에 여러 개의 짧은 예제를 넣어 효율성을 높일 수 있음
packing = False

# GPU 0 전체 모델 로드
device_map = {"": 0}

## 데이터 세트 로딩과 데이터 타입
dataset = load_dataset(dataset_name, split="train")

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
# 모델 계산에 사용될 데이터 타입 결정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # 모델을 4비트로 로드할지 여부를 결정
    bnb_4bit_quant_type=bnb_4bit_quant_type, # 양자화 유형을 설정
    bnb_4bit_compute_dtype=compute_dtype,  # 계산에 사용될 데이터 타입을 설정
    bnb_4bit_use_double_quant=use_nested_quant, # 중첩 양자화를 사용할지 여부를 결정
)


########## GPU 호환성 확인

# 만약 GPU가 최소한 버전 8 이상이라면 (major >= 8) bfloat16을 지원한다고 메시지를 출력.
# bfloat16은 훈련 속도를 높일 수 있는 데이터 타입.

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


########### 베이스 모델 로딩#####
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 동일한 batch 내에서 입력의 크기를 동일하기 위해서 사용하는 Padding Token을 End of Sequence라고 하는 Special Token으로 사용한다.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Padding을 오른쪽 위치에 추가한다.

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM", # 파인튜닝할 태스크를 Optional로 지정할 수 있는데, 여기서는 CASUAL_LM을 지정하였다.
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()

# 훈련이 완료된 모델을 'new_model'에 저장
trainer.model.save_pretrained(new_model)

# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합

model = model.merge_and_unload()

# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"

코드2

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# 1. 모델 로드
model_name = "NousResearch/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. LoRA 설정
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# 3. 학습 파라미터 설정
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_hf",
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    report_to="tensorboard",
)

# 4. 데이터셋 로드 (THUDM/LongWriter-6k 데이터셋 사용)
dataset = load_dataset("THUDM/LongWriter-6k", split="train")
print("dataset.column_names!!!")
print(dataset.column_names)

# 데이터셋 구조 확인
print("Dataset structure:")
print(dataset[0])

# 5. 데이터셋 토크나이즈 함수 정의
def tokenize(element):
    text = str(element['messages'])
    return tokenizer(text, truncation=True, max_length=512)

# 6. 데이터셋 토크나이즈
tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# 데이터 콜레이터 정의
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 7. SFTTrainer로 학습 진행
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    dataset_text_field="messages",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    data_collator=data_collator,
)

trainer.train()

# 8. 모델 저장
trainer.save_model("./trained_model")

# 9. 모델 병합 후 저장
merged_model = model  # 필요에 따라 병합 로직을 추가
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

# 10. 파인튜닝된 모델 테스트
def get_completion(query, model, tokenizer, max_tokens=512):
    # 모델과 입력 데이터를 같은 장치로 이동 (GPU가 사용 가능할 경우 CUDA로 이동)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 입력 데이터를 해당 장치로 이동
    inputs = tokenizer(query, return_tensors="pt").to(device)

    # 모델로 텍스트 생성
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)

    # 결과를 디코딩하여 반환
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 테스트 실행
result = get_completion(
    query="'엣지 태양'에 대한 미드저니 프롬프트",
    model=merged_model,
    tokenizer=tokenizer,
    max_tokens=512,
)
print(result)


Fine-tuned 모델의 사용

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# LoRA 모델 설정 로드
config = PeftConfig.from_pretrained("./trained_model")

# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

# LoRA 가중치 로드
model = PeftModel.from_pretrained(base_model, "./trained_model")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# 모델을 평가 모드로 설정
model.eval()

def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def chat_with_model():
    print("채팅을 시작합니다. 종료하려면 'quit'를 입력하세요.")
    while True:
        user_input = input("사용자: ")
        if user_input.lower() == 'quit':
            break
        response = generate_text(user_input)
        print("모델:", response)

# 대화형 채팅 시작
if __name__ == "__main__":
    chat_with_model()


# 서버 실행 명령 (터미널에서):
# uvicorn main:app --host 0.0.0.0 --port 8000